In [2]:
import pandas as pd
import geopandas as gpd
import json 
from os import path
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, tickers
from bokeh.models.tickers import Ticker, BasicTicker, FixedTicker, LogTicker
from bokeh.palettes import brewer
from bokeh.palettes import magma


from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

# from bokeh.io import output_notebook, show, output_file
# from bokeh.plotting import figure
# from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
# from bokeh.palettes import brewer

def load_data(year):
    year = str(year)
    shapefile = "data/All_V5/%s/scd_glb%s.shp" % (year,year)
    
    gdf = gpd.read_file(shapefile)
    gdf.columns = ['country', 'year', 'rate_scd', 'pop_yr', 'population', 'geometry']
#     gdf.columns = ['country', 'x', 'country_yr', 'org_country', 'year', 'raw_scd', 'population', 'rate_scd', 'HDI', 'GDP', 'GDP_pc', 'geometry']
    gdf['rate_scd_t'] = pow(gdf['rate_scd'], 1).round(4) 
    gdf['rate_scd'] = gdf['rate_scd'].round(4) 
    gdf.fillna('No data', inplace = True)
    gdf.head()

    json_suicide = gdf
    json_suicide.head()

    merged_json = json.loads(gdf.to_json())
    json_data = json.dumps(merged_json)
#     geosource = GeoJSONDataSource(geojson = json_data)
    
    return json_data #geosource

ModuleNotFoundError: No module named 'geopandas'

In [2]:
year = 2000
year = str(year)
shapefile = "data/All_V5/%s/scd_glb%s.shp" % (year,year)
gdf = gpd.read_file(shapefile)
gdf.head()

,sovrgnt,glbl_yr,scd_r_1,pop_yer,pp_smpp,geometry
0,Afghanistan,2000.0,0.102638,2000.0,20093756.0,"POLYGON ((5968437.883624226 6955630.202336617,..."
1,Angola,2000.0,0.117109,2000.0,16440924.0,(POLYGON ((1021086.296115213 -336066.886964715...
2,Albania,2000.0,0.059358,2000.0,3089027.0,"POLYGON ((1199902.154461124 4476351.053550121,..."
3,United Arab Emirates,2000.0,0.042151,2000.0,3154925.0,"POLYGON ((2867427.602064077 2928198.822756103,..."
4,Argentina,2000.0,0.124870,2000.0,37057452.0,(POLYGON ((-5264200.68755088 -4871531.17510359...


In [3]:
years = list(range(1980, 2016, 1))
  
# output_file('cbar.html')

# json_suicide = load_data(1999)

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = load_data(2000))
# geosource = GeoJSONDataSource(geojson = json_data(2016))

#Define a sequential multi-hue color palette.
palette = magma(64)

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 0.5, nan_color = '#ffffff') #'#d9d9d9')

#Define custom tick labels for color bar.
# tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
# tick_labels = {'0': '0%', '10': '10%', '20':'20%', '30':'30%', '40':'40%', '50':'50%', '60':'60%','70':'70%', '80': '>80%'}
# tick_labels = {'0': '0%', '10': '0.01', '20':'0.04', '30':'0.09', '40':'0.16', '50':'0.25', '60':'0.36','70':'0.49', '80': '0.64'}
tick_labels = {'0': '0', '0.1':'0.1', '0.20':'0.2', '30':'0.3','0.4':'0.4', '0.5': '>0.5'}

#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),('% suicide rate (per 1k)', '@rate_scd'),('year', '@year')])


#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)


#Create figure object.
p = figure(title = 'Suicide Rate per1k', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource, fill_color = {'field' :'rate_scd_t', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')


# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = int(slider.value)
    new_data = load_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Suicide Indicidence Rate (per 1k population), %d' %yr
    
# Make a slider object: slider 
# slider = Slider(start=years[0], end=years[-1], value=years[0], step=1, title="Year")
# slider.on_change('value', update_plot)

slider = Slider(title = 'Year',start = 1985, end = 2013, step = 1, value = 2000)
slider.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(layout)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [4]:

# # json_suicide, suicide_new = json_data(2005)


# #Input GeoJSON source that contains features for plotting.
# # geosource = GeoJSONDataSource(geojson = json_suicide)
# year = 2000
# geosource = load_data(year)

# #Define a sequential multi-hue color palette.
# # palette = brewer['YlOrRd'][8]
# palette = magma(64)

# #Reverse color order so that dark blue is highest suicide rate.
# palette = palette[::-1]

# #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
# color_mapper = LinearColorMapper(palette = palette, low = 0, high = 0.5)

# #Define custom tick labels for color bar.

# # tick_label = [str(x/10) for x in range(0, 9, 1)]
# # tick_rate = [str(round(pow(x/10,2),2)) for x in range(0, 9, 1)]
# # tick_labels = dict(zip(tick_label, tick_rate))
# tick_labels = {'0': '0', '1': '0.01', '2':'0.04', '3':'0.09', '4':'0.16', '5':'0.25', '6':'0.36','7':'0.49', '8': '0.64'}

# #Create color bar. 
# color_bar = ColorBar(color_mapper=color_mapper, label_standoff= 8, width = 500, height = 20,
# border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

# #Create figure object.
# p = figure(title = 'Suicide Rate per1k', plot_height = 600 , plot_width = 950, toolbar_location = None)
# p.xgrid.grid_line_color = None
# p.ygrid.grid_line_color = None

# #Add patch renderer to figure. 
# p.patches('xs','ys', source = geosource,fill_color = {'field' :'rate_scd_t', 'transform' : color_mapper},
#           line_color = 'black', line_width = 0.25, fill_alpha = 1)


# #Specify figure layout.
# p.add_layout(color_bar, 'below')

# #Display figure inline in Jupyter Notebook.
# output_notebook()

# #Display figure.
# show(p)

In [5]:
# bokeh serve --show filename.ipynb

In [6]:
'''path '''
# year = 'tmpdir2' + '/'
# shapefile = path.join('data/', year, 'torn.shp')
# gdf = gpd.read_file(shapefile)

'path '

In [7]:
# #Define function that returns json_data for year selected by user.
# import json 

# def json_data(selectedYear):
#     yr = selectedYear
#     suicide_yr = suicide_rename[suicide_rename['year'] == yr]
    
# #     return suicide_yr
#     suicide_new = gdf.merge(suicide_yr.iloc[:, 2:], right_on = 'country_name', left_on = 'country', how = 'left')
#     suicide_new['suicide_rate_transform'] = pow(suicide_new.suicide_rate_per1k, 0.5) *10

#     suicide_new.fillna('No data', inplace = True)
    
#     merged_json = json.loads(suicide_new.to_json())
#     json_data = json.dumps(merged_json)
    
#     return json_data, suicide_new


In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df.head()

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND


In [1]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

output_file("gmap.html")

map_options = GMapOptions(lat=30.2861, lng=-97.7394, map_type="roadmap", zoom=11)

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
p = gmap("GOOGLE_API_KEY", map_options, title="Austin")

source = ColumnDataSource(
    data=dict(lat=[ 30.29,  30.20,  30.29],
              lon=[-97.70, -97.74, -97.78])
)

p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

show(p)

## 